In [ ]:
#pip3 install notebook ipywidgets plotly
#jupyter nbextension enable --py widgetsnbextension
#jupyter nbextension enable --py plotlywidget

In [1]:
import igraph as ig
import pandas as pd
import webbrowser
import json
import urllib.request
import chart_studio.plotly as py
import plotly.graph_objs as go
from ipywidgets import widgets

In [2]:
#Load some data
data = []
req = urllib.request.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
opener = urllib.request.build_opener()
f = opener.open(req)
data = json.loads(f.read())

In [3]:
N = len(data['nodes'])
df_node = pd.DataFrame(data['nodes'])
df_node.drop(columns=['group'],axis=1,inplace=True)
df_node['url'] = 'https://fr.wikipedia.org/wiki/In_Deep'
df_edge = pd.DataFrame(data['links'])
df_edge.drop(columns=['value'],axis=1,inplace=True)

In [4]:
df_node.head()

,name,url
0,Myriel,https://fr.wikipedia.org/wiki/In_Deep
1,Napoleon,https://fr.wikipedia.org/wiki/In_Deep
2,Mlle.Baptistine,https://fr.wikipedia.org/wiki/In_Deep
3,Mme.Magloire,https://fr.wikipedia.org/wiki/In_Deep
4,CountessdeLo,https://fr.wikipedia.org/wiki/In_Deep


In [5]:
df_edge.head()

,source,target
0,1,0
1,2,0
2,3,0
3,3,2
4,4,0


In [6]:
#Extract list
labels = df_node['name'].tolist()
color_node = [0.0 for i in range(len(labels))]
urls = df_node['url'].tolist()
Edges = df_edge[['source','target']].values.tolist()

G = ig.Graph(Edges, directed=False)
layt = G.layout('kk', dim=3)

In [7]:
#Coordinates for the nodes and edges
Xn = [layt[k][0] for k in range(N)]
Yn = [layt[k][1] for k in range(N)]
Zn = [layt[k][2] for k in range(N)]
Xe = []
Ye = []
Ze = []
for e in Edges:
    Xe += [layt[e[0]][0],layt[e[1]][0], None]
    Ye += [layt[e[0]][1],layt[e[1]][1], None]
    Ze += [layt[e[0]][2],layt[e[1]][2], None]

In [8]:
#Plot the edges
trace1 = go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
#Plot the nodes
trace2 = go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=color_node,
                             colorscale='YlOrRd',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text',
            customdata=urls
       )

axis = dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Wikipedia Recommender System",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text='By clicking on a node, you will be directed on the corresponding web page.',
            xref='paper',
            yref='paper',
            x=0,
            y=0,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],
  )

g = go.FigureWidget(data=[trace1, trace2],layout=layout)

In [9]:
#Text Box for the query
textbox = widgets.Text(
    value='',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

#On a new query, compute the predictions and color the nodes accordingly
def response(change):
    query = textbox.value
#     preds = make_prediction(query)
#     dict_colors = compute_color(preds)
# key is node index, and value is an integer for the color
    dict_colors = {10:0.2,11:0.2}
    for k,v in dict_colors.items():
        color_node[k] = v
        
    with g.batch_update():
        g.data[1].marker.color = color_node
        
textbox.observe(response, names="value")

In [10]:
# Open url when clicking on node
def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        url = g.data[1].customdata[points.point_inds[0]]
        webbrowser.open_new_tab(url)

g.data[1].on_click(update_point)

In [11]:
container = widgets.HBox(children=[textbox])
widgets.VBox([container, g])